In [1]:
import pandas as pd
import numpy as np
import seaborn as sb
import matplotlib.pyplot as plt
import plotly.express as px
import csv
import statistics
from pandas.api.types import CategoricalDtype

In [2]:
# Sequence class labels for data sets

names = open("cnames.tsv")
names1 = csv.reader(names, delimiter="\t")
seq_names = pd.DataFrame(names1)

new_header = seq_names.iloc[0]
seq_names= seq_names[1:] 
seq_names.columns = new_header 
seq_names['ID'] = seq_names['ID'].astype(str) + " " + seq_names['name'].astype(str)
seq_names['index'] = seq_names['index'].astype(int)

seq_names_original = seq_names.sort_values(by = "index")

labels = (seq_names_original["ID"].astype(str)).tolist()
labels_reordered = (seq_names["ID"].astype(str)).tolist()

In [3]:
# Load and Set-up Breast Cancer Dataset

data = pd.DataFrame(np.load("breast_cancer.finemapping.sc_scores.npy")).iloc[:, 0:40]
data.columns = labels
data = data[labels_reordered]
bc_labels = csv.reader(open("breast_cancer.finemapping.row_labels.txt"), delimiter="\t")
bc_labels1 = pd.DataFrame(bc_labels)
new_header1 = bc_labels1.iloc[0]
bc_labels1 = bc_labels1[1:]
bc_labels1.columns = new_header1
bc_labels1 = bc_labels1.reset_index()
df1 = bc_labels1.join(data, how='outer').set_index('name').iloc[:, 1:]
df1.index.name = None

# # Take averages of identital mutations
# row_labels = df1.index
# df1['name'] = df1.index

# df1 = (df1.groupby((df1['name'] != df1['name'].shift()).cumsum())
#                   .mean()
#                   .reset_index(drop=True))
# df1.index = pd.unique(row_labels)

In [ ]:
df1   

In [ ]:
sub_df = df1[df1['chrom'].str.endswith('chr' + str(i))].iloc[:, 1:]

row_labels = sub_df.index
sub_df['name'] = sub_df.index
sub_df = (sub_df.groupby((sub_df['name'] != sub_df['name'].shift()).cumsum())
                  .mean()
                  .reset_index(drop=True))
sub_df.index = pd.unique(row_labels)

In [ ]:
df2 = sub_df.stack().reset_index().iloc[:, ::-1]
df2.columns = ['vals', 'profiles', 'mutation']

In [4]:
def scatter_plot_vis(xlabel, ylabel, color):
    fig = px.scatter(x = xlabel, y = ylabel, color = color)
    fig.show()

In [ ]:
for i in range(1):
    sub_df = df1[df1['chrom'].str.endswith('chr' + str(i + 1))].iloc[:, 1:]

    row_labels = sub_df.index
    sub_df['name'] = sub_df.index
    sub_df = (sub_df.groupby((sub_df['name'] != sub_df['name'].shift()).cumsum())
                      .mean()
                      .reset_index(drop=True))
    sub_df.index = pd.unique(row_labels)
    df2 = sub_df.stack().reset_index().iloc[:, ::-1]
    df2.columns = ['vals', 'profiles', 'mutation']
    
    xlabel = df2['mutation'].tolist()
    ylabel = df2['vals'].tolist()
    color = df2['profiles'].tolist()
    
    fig = px.scatter(x = xlabel, y = ylabel, color = color)
    fig.show()
    